In [16]:
import pandas as pd
import matplotlib
import numpy as np
from sklearn import preprocessing
from sklearn import neural_network
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [32]:
df = pd.read_csv('../data/aggregated/processed_data/data-4-7-18-full.csv')

class PreprocessCars:
    """Code to preprocess the dataframe in preparation for training"""
    def get_train_test(self, orig_df):
        # Replace missing continuous values
        df = orig_df.copy(deep=True)
        df['mileage'] = df['mileage'].fillna((df['mileage'].mean()))
        df['year'] = df['year'].fillna((df['year'].mean()))
        
        # Replace missing categorical values
        df['make'] = df['make'].fillna(df['make'].value_counts().index[0])
        df['model'] = df['model'].fillna(df['model'].value_counts().index[0])
        df['state'] = df['state'].fillna(df['state'].value_counts().index[0])
        df['transmission'] = df['transmission'].fillna(df['transmission'].value_counts().index[0])
        
        # Normalize the continuous values
        df['mileage'] = (df['mileage']-df['mileage'].mean())/df['mileage'].std() # This approach puts between 0 and 1, is that an issue?
        df['year'] = (df['year']-df['year'].mean())/df['year'].std()
        
        # Only select a subset of features
        features = df[['make', 'model', 'mileage', 'state', 'transmission', 'year']]
        labels = df[['price']]
        
        # TODO - encode categorical features
        features_encoded = pd.get_dummies(features, columns=['make', 'model', 'state', 'transmission'])
        X_train, X_test, Y_train, Y_test = train_test_split(features_encoded, labels, test_size=0.2, train_size=0.8)
        return X_train, X_test, Y_train, Y_test


In [33]:
preprocessor = PreprocessCars()
X_train, X_test, Y_train, Y_test = preprocessor.get_train_test(df)
X_train.head(1)

# features_encoded = pd.get_dummies(features, columns=['make', 'model', 'state', 'transmission'])
# label_encoder = LabelEncoder()
# label_encoder.fit_transform(features)
# enc = OneHotEncoder()
# features_encoded = enc.fit_transform(features)
# label_encoded_features.head(1)
#X_train, X_test, Y_train, Y_test = train_test_split(features_encoded, labels, test_size=0.2, train_size=0.8)

,mileage,year,make_Acura,make_Aston Martin,make_Audi,make_BMW,make_Bentley,make_Buick,make_Cadillac,make_Chevrolet,...,state_Ut,state_Utah,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,transmission_Automatic,transmission_Manual
37015,-0.420357,0.22652,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
mlp_model = neural_network.MLPRegressor(hidden_layer_sizes=(100,100))
mlp_model.fit(X_train, Y_train)

/users/guest/d/dsw88/.virtualenvs/cs478/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/users/guest/d/dsw88/.virtualenvs/cs478/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [35]:
mlp_model.score(X_test, Y_test)

0.8745887985884477

# Save Model
Dump the preprocessor class and trained model:

In [41]:
import pickle
with open('../webapp/feature-preprocessor.pkl', 'wb') as preprocessorFile:
    pickle.dump(preprocessor, preprocessorFile)
with open('../webapp/trained-model.pkl', 'wb') as modelFile:
    pickle.dump(mlp_model, modelFile)